In [12]:
import pandas as pd
import os

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df.head()

,Days_Since_Start,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Volume,TSLA_RSI_7,TSLA_RSI_14,TSLA_MACD,TSLA_MACD_Signal,...,NEE_MACD_Signal,HD_Open,HD_High,HD_Low,HD_Close,HD_Volume,HD_RSI_7,HD_RSI_14,HD_MACD,HD_MACD_Signal
0,196,1.424667,1.483333,1.394667,1.460667,37297500.0,92.220196,43.408794,-0.064685,-0.083391,...,0.059857,19.278965,19.349948,19.058918,19.215080,17061600,25.229445,25.954283,-0.802271,-0.824908
1,197,1.456667,1.456667,1.336667,1.353333,27379500.0,71.260949,39.206237,-0.058209,-0.078355,...,0.071355,18.987938,19.896520,18.895661,19.825537,23279400,44.368562,43.190715,-0.756203,-0.811167
2,198,1.377333,1.393333,1.300000,1.348000,18787500.0,74.198423,39.159154,-0.052897,-0.073263,...,0.079636,19.719066,19.882326,19.413839,19.499018,17161300,37.425292,44.047680,-0.737539,-0.796441
3,199,1.366667,1.416667,1.358000,1.400000,14367000.0,72.916658,46.915168,-0.043984,-0.067407,...,0.087182,19.669378,20.208849,19.669378,20.031391,26142700,44.414890,52.659520,-0.672043,-0.771562
4,200,1.412667,1.437333,1.404000,1.419333,9804000.0,65.010249,57.983173,-0.034957,-0.060917,...,0.093594,19.974605,20.130768,19.804246,20.052687,18979200,49.562788,54.446405,-0.611372,-0.739524


In [13]:
import pandas as pd

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Days_Since_Start'])
df.set_index('Days_Since_Start', inplace=True)

# 确保其他列是数值类型，去掉可能的字符串类型列（例如日期）
df = df.apply(pd.to_numeric, errors='coerce')

# 检查数据的头部
print(df.head())

                  TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  TSLA_Volume  \
Days_Since_Start                                                            
196                1.424667   1.483333  1.394667    1.460667   37297500.0   
197                1.456667   1.456667  1.336667    1.353333   27379500.0   
198                1.377333   1.393333  1.300000    1.348000   18787500.0   
199                1.366667   1.416667  1.358000    1.400000   14367000.0   
200                1.412667   1.437333  1.404000    1.419333    9804000.0   

                  TSLA_RSI_7  TSLA_RSI_14  TSLA_MACD  TSLA_MACD_Signal  \
Days_Since_Start                                                         
196                92.220196    43.408794  -0.064685         -0.083391   
197                71.260949    39.206237  -0.058209         -0.078355   
198                74.198423    39.159154  -0.052897         -0.073263   
199                72.916658    46.915168  -0.043984         -0.067407   
200             

In [14]:
rows, cols = df.shape
print(f"DataFrame 共有 {rows} 行, {cols} 列")


DataFrame 共有 2633 行, 91 列


In [15]:
# 检查数据的列名
print(df.columns)

# 检查是否有缺失值
print(df.isnull().sum())


Index(['TSLA_Open', 'TSLA_High', 'TSLA_Low', 'TSLA_Close', 'TSLA_Volume',
       'TSLA_RSI_7', 'TSLA_RSI_14', 'TSLA_MACD', 'TSLA_MACD_Signal',
       'AAPL_Open', 'AAPL_High', 'AAPL_Low', 'AAPL_Close', 'AAPL_Volume',
       'AAPL_RSI_7', 'AAPL_RSI_14', 'AAPL_MACD', 'AAPL_MACD_Signal', 'GE_Open',
       'GE_High', 'GE_Low', 'GE_Close', 'GE_Volume', 'GE_RSI_7', 'GE_RSI_14',
       'GE_MACD', 'GE_MACD_Signal', 'QQQ_Open', 'QQQ_High', 'QQQ_Low',
       'QQQ_Close', 'QQQ_Volume', 'QQQ_RSI_7', 'QQQ_RSI_14', 'QQQ_MACD',
       'QQQ_MACD_Signal', 'NVDA_Open', 'NVDA_High', 'NVDA_Low', 'NVDA_Close',
       'NVDA_Volume', 'NVDA_RSI_7', 'NVDA_RSI_14', 'NVDA_MACD',
       'NVDA_MACD_Signal', 'UNH_Open', 'UNH_High', 'UNH_Low', 'UNH_Close',
       'UNH_Volume', 'UNH_RSI_7', 'UNH_RSI_14', 'UNH_MACD', 'UNH_MACD_Signal',
       'CAT_Open', 'CAT_High', 'CAT_Low', 'CAT_Close', 'CAT_Volume',
       'CAT_RSI_7', 'CAT_RSI_14', 'CAT_MACD', 'CAT_MACD_Signal', 'AMZN_Open',
       'AMZN_High', 'AMZN_Low', 'AMZN_

In [16]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class TradingEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}
    
    def __init__(self, df, window_size=10, initial_balance=10000, max_drawdown=0.2, risk_free_rate=0.01, transaction_cost=0.001):
        super(TradingEnv, self).__init__()
        
        self.df = df
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.max_drawdown = max_drawdown
        self.risk_free_rate = risk_free_rate
        self.transaction_cost = transaction_cost  # 每笔交易的手续费比例
        
        # 获取股票代码列表
        self.tickers = sorted(set(col.split('_')[0] for col in df.columns if '_' in col))
        self.num_stocks = len(self.tickers)
        self.features_per_stock = 9  # 9个特征 (Open, High, Low, Close, Volume, RSI_7, RSI_14, MACD, MACD_Signal)
        
        # 定义状态空间 (window_size 天 * 10 只股票 * 9 特征 + 账户状态)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size * 90,), dtype=np.float32)

        # 动作空间 (-1 ~ 1 代表买入 / 卖出比例)
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.num_stocks,), dtype=np.float32)

        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = np.zeros(self.num_stocks)  # 每只股票的持仓
        self.total_profit = 0
        self.peak_value = self.initial_balance  # 初始的峰值（用于回撤计算）
        self.done = False
        self.history = []  # 资产历史
        
        return self._next_observation(), {}
    
    def _next_observation(self):
        obs = self.df.iloc[self.current_step - self.window_size:self.current_step, 1:].values.flatten()  # 跳过 Days_Since_Start
        return obs

        
    def step(self, action):
        if self.done:
            return self._next_observation(), 0, True, False, {}

        close_prices = self.df.iloc[self.current_step, [self.df.columns.get_loc(f"{t}_Close") for t in self.tickers]]
        
        # 计算当前总资产（现金 + 股票市值）
        total_value = self.balance + np.sum(self.shares_held * close_prices.values)
        
        # 计算目标持仓金额（clip 避免负值，防止卖空）
        target_values = np.clip(action, 0, 1) * total_value

        # 计算目标持仓的股数
        target_shares = target_values / close_prices.values
        
        # 计算买入/卖出的股数
        delta_shares = target_shares - self.shares_held
        
        # 执行交易
        for i in range(self.num_stocks):
            current_price = close_prices[i]
            cost = abs(delta_shares[i]) * current_price * self.transaction_cost  # 交易成本

            if delta_shares[i] > 0:  # 需要买入
                max_shares = self.balance // (current_price + cost)  # 计算最多能买多少股
                buy_shares = min(max_shares, delta_shares[i])  # 限制买入数量
                self.shares_held[i] += buy_shares
                self.balance -= buy_shares * current_price + cost

            elif delta_shares[i] < 0:  # 需要卖出
                sell_shares = min(abs(delta_shares[i]), self.shares_held[i])  # 不能卖空
                self.shares_held[i] -= sell_shares
                self.balance += sell_shares * current_price - cost

        # 计算新资产总值
        total_value = self.balance + np.sum(self.shares_held * close_prices.values)
        prev_total_value = self.history[-1] if len(self.history) > 1 else self.initial_balance
        daily_return = (total_value - prev_total_value) / prev_total_value  # 日收益率
        self.total_profit = total_value - self.initial_balance

        # 计算回撤
        self.peak_value = max(self.peak_value, total_value)
        drawdown = (self.peak_value - total_value) / self.peak_value if self.peak_value > 0 else 0

        # 计算夏普比率
        self.history.append(total_value)
        returns = np.diff(self.history[-min(len(self.history), 30):]) / self.history[-min(len(self.history), 30):-1]
        sharpe_ratio = (np.mean(returns) - self.risk_free_rate) / (np.std(returns) + 1e-6) if len(returns) > 1 else 0

        # 奖励函数
        reward = (
            daily_return * 100  # 放大日收益对奖励的贡献
            - drawdown * 5  # 持续惩罚回撤，而不是等超过阈值才惩罚
            + np.sign(self.total_profit) * np.log1p(abs(self.total_profit)) * 0.1  # 让利润增长时奖励逐步增加
            + sharpe_ratio * 0.1  # 适度考虑夏普比率，但不让它主导
            )


        # 结束条件
        self.current_step += 1
        if self.current_step >= len(self.df) - 1:
            self.done = True
        
        elif drawdown >= 0.5:  # 若回撤达到 50%，提前结束
            self.done = True


        return self._next_observation(), reward, self.done, False, {}


    
    def render(self, mode="human"):
        print(f'Step: {self.current_step}, Balance: {self.balance:.2f}, Shares: {self.shares_held}, Profit: {self.total_profit:.2f}, Drawdown: {self.peak_value - (self.balance + np.sum(self.shares_held * self.df.iloc[self.current_step][[f"{t}_Close" for t in self.tickers]])):.2f}')

print("O_o!")

O_o!


In [17]:
import gymnasium as gym
import numpy as np

# 假设 df 是你已经加载并预处理好的 DataFrame，包含所有需要的特征
# df = pd.read_csv('your_stock_data.csv')

# 创建环境实例
env = TradingEnv(df)  # 使用你的数据

# 测试环境初始化
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# 测试 reset 和 _next_observation
obs, info = env.reset()
print("Initial Observation:", obs)


Observation Space: Box(-inf, inf, (900,), float32)
Action Space: Box(-1.0, 1.0, (10,), float32)
Initial Observation: [ 1.48333299e+00  1.39466703e+00  1.46066701e+00  3.72975000e+07
  9.22201958e+01  4.34087939e+01 -6.46850475e-02 -8.33913485e-02
  7.51978817e+00  7.51978817e+00  7.21042614e+00  7.39038515e+00
  1.02447800e+09  1.41132493e+01  2.77689318e+01 -6.74227941e-02
 -2.62855064e-02  5.17271744e+01  5.20804997e+01  5.11265131e+01
  5.16565094e+01  1.02396750e+07  4.15999081e+01  4.42771895e+01
 -9.42785420e-01 -1.24506051e+00  3.90346988e+01  3.93331427e+01
  3.87099192e+01  3.92541428e+01  8.18008000e+07  5.85526974e+01
  4.73683933e+01 -1.46861507e-01 -2.96142639e-01  2.33627513e-01
  2.40734902e-01  2.32022593e-01  2.39817828e-01  7.98440000e+08
  5.35545959e+01  4.24170542e+01 -1.02271605e-02 -1.21284987e-02
  2.44886278e+01  2.48742751e+01  2.43359767e+01  2.47617950e+01
  8.57550000e+06  7.56758722e+01  6.51206712e+01  9.71865263e-02
 -5.71588427e-02  4.45485893e+01  4.51

In [18]:
# 测试执行一步操作
action = np.random.uniform(-1, 1, size=(env.num_stocks,))  # 随机生成动作
print("Test Action:", action)

# 进行一步操作并获取返回的观察值、奖励等
obs, reward, done, truncated, info = env.step(action)
print("New Observation:", obs)
print("Reward:", reward)
print("Done:", done)


Test Action: [-0.80653338 -0.62584283  0.15837581 -0.01700996  0.62821354  0.73470086
  0.12031544  0.77485152  0.91070403 -0.52011036]
New Observation: [ 1.45666695e+00  1.33666694e+00  1.35333300e+00  2.73795000e+07
  7.12609494e+01  3.92062370e+01 -5.82086295e-02 -7.83548047e-02
  7.30973612e+00  7.61067180e+00  7.22276563e+00  7.58027697e+00
  1.07495080e+09  3.25979834e+01  4.52739356e+01 -6.54673241e-02
 -3.41218700e-02  5.07378612e+01  5.30344895e+01  5.07378612e+01
  5.27871590e+01  1.40386600e+07  4.96553316e+01  5.73718569e+01
 -8.47155793e-01 -1.16547957e+00  3.86221417e+01  3.97281380e+01
  3.84992537e+01  3.97281380e+01  8.90612000e+07  6.01264421e+01
  6.52173061e+01 -9.48661735e-02 -2.55887346e-01  2.33627499e-01
  2.47154484e-01  2.31564043e-01  2.45778874e-01  1.16098800e+09
  5.79399830e+01  5.31089041e+01 -9.26364943e-03 -1.15555288e-02
  2.51876161e+01  2.53000962e+01  2.42958053e+01  2.47698307e+01
  1.34939000e+07  7.21651289e+01  7.27274288e+01  1.37372147e-01
 -

/tmp/ipykernel_12043/2745502066.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = close_prices[i]


In [19]:
# 测试多步操作
for i in range(10):
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}, Done: {done}")
    if done:
        break


Step 1, Reward: -2.9293071316721884, Total Profit: -227.262614592506, Done: False
Step 2, Reward: -0.46419129455463604, Total Profit: -195.81970063779045, Done: False
Step 3, Reward: -1.2626811282461265, Total Profit: -236.90221945086887, Done: False
Step 4, Reward: -1.379504529356335, Total Profit: -283.2598180104833, Done: False
Step 5, Reward: -0.9740324648922676, Total Profit: -289.57885925069604, Done: False
Step 6, Reward: -2.3923529106831856, Total Profit: -421.78891237759854, Done: False
Step 7, Reward: -4.760567970222585, Total Profit: -761.6955787634106, Done: False
Step 8, Reward: -1.4468855891814008, Total Profit: -782.4225166913693, Done: False
Step 9, Reward: -2.21442728947883, Total Profit: -868.0597889994915, Done: False
Step 10, Reward: -1.3650930450850134, Total Profit: -875.3002901175132, Done: False


/tmp/ipykernel_12043/2745502066.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = close_prices[i]


In [20]:
# 测试环境是否能正常结束
env.reset()
while not env.done:
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Balance: {env.balance}, Total Profit: {env.total_profit}")


Balance: 28.6116497044778, Total Profit: -28.308434856946405
Balance: 20.403081274377442, Total Profit: -20.7869417103866
Balance: 15.951752693990997, Total Profit: 40.262704916882285
Balance: 1271.0869413809285, Total Profit: 9.455715392725324
Balance: 5914.194260868276, Total Profit: 2.136510023745359
Balance: 2858.9939776112924, Total Profit: -34.70561411418021
Balance: -1.079033165889062, Total Profit: -47.605996698001036
Balance: -3.613391084183406, Total Profit: -411.44308575175273
Balance: 692.6969891969093, Total Profit: -453.28102743670934
Balance: 10.247396018883983, Total Profit: -540.3154873581916
Balance: 232.95783639982096, Total Profit: -547.6377463434637
Balance: 1755.3154055077484, Total Profit: -434.85024112372594
Balance: 7390.198186308695, Total Profit: -438.2379664168675
Balance: 1980.8970612951427, Total Profit: -490.3817431011066
Balance: 6099.715364602965, Total Profit: -504.49903237478793
Balance: 570.0317797143589, Total Profit: -557.6666363542245
Balance: 648

/tmp/ipykernel_12043/2745502066.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = close_prices[i]


Balance: 763.2797535587665, Total Profit: -444.42815817104565
Balance: 1390.8854671217814, Total Profit: -232.60253851634116
Balance: 2595.867452225485, Total Profit: -271.55865030366476
Balance: 393.2732564806238, Total Profit: -281.28139411556185
Balance: 2985.724640904104, Total Profit: -352.7447205418666
Balance: 2854.7869988408465, Total Profit: -416.35381779782256
Balance: 3298.600637615054, Total Profit: -410.05558611809465
Balance: 355.81104533023904, Total Profit: -413.1802433615194
Balance: 73.65969797188808, Total Profit: -174.81861464672147
Balance: 2646.3124226295463, Total Profit: -233.59768173969678
Balance: 7023.614369699092, Total Profit: -246.51578201976372
Balance: 2916.822579941952, Total Profit: -259.316916914353
Balance: 393.63925499488937, Total Profit: -227.3877983383336
Balance: 2371.385354209072, Total Profit: -274.0771680011421
Balance: 378.98241489390205, Total Profit: -227.00867701412244
Balance: 594.2275952289206, Total Profit: -176.33187423204618
Balance:

In [21]:
from stable_baselines3 import PPO

# 创建模型
model = PPO("MlpPolicy", env, verbose=1)

# 训练模型
model.learn(total_timesteps=10000)

# 测试训练结果
obs, _ = env.reset()
for i in range(10):  # 运行10步
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}")
    if done:
        break


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/tmp/ipykernel_12043/2745502066.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = close_prices[i]


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 267      |
|    ep_rew_mean     | -594     |
| time/              |          |
|    fps             | 370      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 269         |
|    ep_rew_mean          | -600        |
| time/                   |             |
|    fps                  | 329         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012078086 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | -0.0103     |
|    learning_rate        | 0.

In [22]:
from sb3_contrib import RecurrentPPO

# 使用 LSTM 版本的策略
model = RecurrentPPO("MlpLstmPolicy", env, verbose=1)

# 训练模型
model.learn(total_timesteps=1000000)

# 保存模型
model.save("recurrent_ppo_trading")

# 测试训练结果
obs, _ = env.reset()
for i in range(10):  # 运行10步
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}")
    if done:
        break


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/tmp/ipykernel_12043/2745502066.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = close_prices[i]


----------------------------
| time/              |     |
|    fps             | 284 |
|    iterations      | 1   |
|    time_elapsed    | 0   |
|    total_timesteps | 128 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 22          |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 256         |
| train/                  |             |
|    approx_kl            | 0.008451212 |
|    clip_fraction        | 0.0359      |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | -0.0588     |
|    learning_rate        | 0.0003      |
|    loss                 | 198         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0288     |
|    std                  | 0.999       |
|    value_loss           | 418         |
-----------------------------------------

KeyboardInterrupt: 